In [1]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
#df$tally_q <- as.integer(df$tally_q)
# df$sale <- as.integer(df$sale)
# df$date <- as.Date(df$date, origin = "1899-12-30")

df$cf_q <- as.integer(rep(0, nrow(df)))
df$cf_b <- as.numeric(rep(0, nrow(df)))
df$sell_q <- as.integer(rep(0, nrow(df)))
df$sell_p <- as.numeric(rep(0, nrow(df)))
df$index <- seq(1:nrow(df))


head(df,3)
tail(df,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,0,0,0,0,1
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.4,50,-5512.4,-1000,0,0,0,0,2
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
39,5/17/2023,Buy,23,16,AMC,1000,5.01,5009.0,2100,5009.0,0,0,0,0,0,39
40,5/30/2023,Sell,23,17,AMC,-1000,4.58,-4582.8,1100,-4582.8,-1000,0,0,0,0,40
41,45079.00,Sell,23,18,AMC,-1100,4.54,-4994.0,0,-4994.0,-1100,0,0,0,0,41


In [2]:
# create seperate buy / sell dfs

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  # Filter sells
df_pl <- data.frame()

In [3]:
# create vars
buy_qty <- buys[1, 'quan']
sell_qty <- abs(sells[1, 'quan'])

buy_basis <- buys[1, 'basis']
sell_basis <- abs(sells[1, 'basis'])


# sell_quan_remainder <- buy_qty - abs(sell_qty)

cat('sell_qty : ', sell_qty, "\n")
cat('buy_qty : ', buy_qty, "\n")
cat('buy_basis :', buy_basis, "\n")
cat('sell_basis :', sell_basis, "\n")
# cat('remainder :',sell_quan_remainder , "\n")

# zero out vars
sells[1, 'tally_q'] <- 0
sells[1, 'tally_b'] <- 0
sells[1, 'Sale'] <- 0

sell_qty :  1000 
buy_qty :  1050 
buy_basis : 4273.5 
sell_basis : 5512.4 


In [4]:
head(buys, 3)
head(sells, 3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,0,0,0,0,1
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,0,0,0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index,Sale
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,0,0.00,-1000,0,0,0,0,2,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,0,0,0,0,4,NA
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,0,0,0,0,6,NA



## <font color = yellow> Keeper for concept above

In [5]:
df_sales <- data.frame()

# add index column

df$index <- seq(1, nrow(df))

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

### <font color= yellow> # Review for user prompt 

In [6]:
# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# str(sell_quan)
# str(sell_price)


In [7]:
for(i in 1:4) {

repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            sell_quan <- df[i, 'quan']
            sell_price <- df[i, 'pps']
            return(sell_quan)
        }
        i <- i + 1
     }
}

Got a Sell! 
Row number  2  Selling  1000 quantity of shares.  
Index number is:  2 


In [8]:


nrow(sells)

     if(buys[1, 'quan'] > sell_quan) {       
         
     # create New Balance
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
    } else if (buys[1, 'quan']  < sell_quan) {       
         cf1 <- buys[1,'quan']
         cf2 <- buys[1, "basis"]
         buys <- buys[-1, ]
         buys[1,'quan'] <- buys[1,'quan'] + cf1
         buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
         }
    


nrow(sells)
head(buys, 3)     
head(df_sales)


[1] 18

[1] 17

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,2050,4.07,8343.5,-1000,4273.5,-5510,0,0,0,0,1
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,0,0,0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,2050,4.07,8343.5,-1000,4273.5,-5510,0,0,0,0,1


In [9]:
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,0,0,0,0,4
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,0,0,0,0,6
8,2/3/2023,Sell,4,4,AMC,-1000,6.19,-6190.10,1050,-6190.10,-1000,0,0,0,0,8


In [10]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# nrow(sells)




#      if(buys[1, 'quan'] > sell_quan) {       
         
#      # create New Balance
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#     } else if (buys[1, 'quan']  < sell_quan) {       
#          cf1 <- buys[1,'quan']
#          cf2 <- buys[1,"basis"]
#          buys <- buys[-1, ]
#          buys[1,'quan'] <- buys[1,'quan'] + cf1
#          buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#          }
    
    

# nrow(sells)
# head(buys, 3)     
# head(df_sales)


In [11]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
df$index <- seq(1:nrow(df))
df$cf_q <- as.integer(rep(0, nrow(df)))
df$cf_b <- as.numeric(rep(0, nrow(df)))
df$sell_q <- as.integer(rep(0, nrow(df)))
df$sell_p <- as.numeric(rep(0, nrow(df)))
df$index <- seq(1:nrow(df))




df[1, 'type'] <- "Buy"

df_sales <- data.frame()
buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

In [12]:
buy_ctr <- as.integer(1)
head(buys,3)
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,1,0,0,0,0
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,3,0,0,0,0
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,5,0,0,0,0


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,50,-5512.40,-1000,2,0,0,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0,0,0
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,6,0,0,0,0


### <font color = lime> Strange result

In [13]:
# df[1, 'type'] <- "Buy"
# head(df,3)

In [14]:
df[i,]

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.4,50,-5512.4,-1000,2,0,0,0,0


## <font color = yellow> Change: don't delete rows, use the new ctr var in the df

In [15]:
# Gets sell information for conditionals 

get_sell <- function () {

    for(i in 1:nrow(sells)) {
    ct <- 0
    repeat {
      
        if(df[i, "type"] == "Sell"){             
            # cat("Got a Sell!", "\n")
            # cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            # cat("Index number is: ", df[i, "index"], "\n")
            sell_index <<- df[i, "index"]
            sell_quan <<- df[i, 'quan']
            return()
        }  # if
        i <- i + 1
     }   # repeat
  } # for
}  # fct
    

In [16]:

for(i in 1:8) {

if(df[i, 'type'] == "Buy" && df[i+1, 'type'] == "Sell") {     
    cat("works with these two rows. One is: " , i, ", Two is: ", i+1, "\n")    
    }
get_sell()

    if(sell_quan >= df[i, 'quan']) {
        
        if(sell_quan == df[i, 'quan']) {
            
            temp <- df[i,]
            df_sales <- rbind(df_sales, temp)
            sells <- sells[-i, ]
            
            }
         
        
        }
    
# test quanity in each row
    
    
    
#if(sells_index - buys_index == 1 && df[i, 'quan'] > 0)
    
 }

works with these two rows. One is:  1 , Two is:  2 
works with these two rows. One is:  3 , Two is:  4 
works with these two rows. One is:  5 , Two is:  6 
works with these two rows. One is:  7 , Two is:  8 


In [17]:
length(buys)
length(sells)

[1] 16

[1] 16

In [18]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

sell_index <- as.integer()
sell_quan <- as.integer()

get_sell()

cat("sell_index is: ", sell_index, "\n")
cat("sell_quan is: ", sell_quan, "\n")

ct <- sell_index


for(i in 1:nrow(df)) {
    
    buy_index <- i
    
    cat("top of for loop, i ia: ", i, "\n") 
    
    cat("buys and sells indexes' are : ", buy_index, " and ", sell_index, "\n")

    if(sell_index - buy_index == 1 && abs(df[i+1, 'quan']) == df[i, 'quan']) {
       cat("This will be a simple wash sale with no c/f", "\n")
       # one possibility of eliminating these rows is to zero the quans and set a flag
        df[i, 'sell_q' ] <- sell_quan
        df[i, 'sell_p'] <- df[i+1, 'pps'] * sell_quan
        df[i +1, 'type'] <- "Sold"
        df[i +1, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("Clean sale. No C/F from sales condition type 1 ", "\n")
        return()       
        
    }
    
    else if(sell_index - buy_index == 1 && df[i, 'quan'] < 0) {
        cat("This will be a simple wash sale with no c/f", "\n")
        }
         
    else if(sell_index - buy_index ==1 && df[i, 'quan']  < df[i+1, 'quan'] ) {
        
        cat("This buy row gets entirely sold with additional sold from the next buy line(s) ", "\n")
         
        # cat("top of if # 1 ", "\n") 
        ct <- ct + buy_ctr   
         
         # one possibility of eliminating these rows is to zero the quans and set a flag
        df[i, 'sell_q' ] <- df[i, 'quan']
        df[i, 'sell_p'] <- df[i+1, 'pps'] * sell_quan
        df[i +1, 'type'] <- "Sold"
        df[i, 'type'] <- "Sold"
        df[i +1, 'index'] <- i
        df[i, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("clean, no C/F from sales condition type 2 ", "\n")
        return()       
        
                
    
         
        # when the buys row i has quanity < sell quan, need carry values forward to next row,
         # And mo
       } else if(sell_index - buy_index ==1 && df[i, 'quan']  > df[i+1, 'quan'] ) {
        
        cat("top of else if # 1 ", "\n") 
        
        
        
        # create allocation and c/f amounts
        
        
        
        df[i, 'cf_q'] <-     df[i, 'quan'] - abs(df[i+1, 'quan'])                                            
        df[i + 1, 'quan'] <- df[i, 'quan'] - abs(df[i+1, 'quan'])  # adj sell quan after removing partial sell
                                             
        df[i, 'cf_b'] <-    df[i, 'cf_q']  * df[i, 'pps']        
        df[i, 'sell_q' ] <- df[i, 'quan'] - df[i, 'cf_q']
        
        df[i, 'sell_p'] <- df[i+1, 'pps'] * (df[i, 'quan'] - abs(df[i+1, 'quan']) )
        df[i , 'type'] <- "Sold"
        df[i+1, 'type'] <- "Part Sold"
        
        df[i +1, 'index'] <- i
        df[i, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("Partial sale. Has C/F from sales condition type 3 ", "\n")
        return()       
        
         ct <- ct + i
        
         buys[i,'sale'] <- buys[i,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[i, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-i, ]
         }
    
   # print(buys[i,])

    }
    
    


head(df, 4)
head(df_sales

NULL

sell_index is:  2 
sell_quan is:  -1000 
top of for loop, i ia:  1 
buys and sells indexes' are :  1  and  2 
top of else if # 1  
Partial sale. Has C/F from sales condition type 3  
[1] "--------"
i value is:  1 
[1] "--------"
[1] 2
[1] "--------"


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,12/31/2022,Sold,1,0,AMC,1050,4.07,4273.50,1050,4273.50,0,1,50,203.5,1000,5510
2,1/27/2023,Part Sold,1,1,AMC,50,5.51,-5512.40,50,-5512.40,-1000,1,0,0.0,0,0
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.00,1050,5625.00,0,3,0,0.0,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0.0,0,0


In [19]:
nrow(df)



[1] 41

In [20]:
stopper

ERROR: Error in eval(expr, envir, enclos): object 'stopper' not found


## <font color = yellow> Working code blocks below:


In [ ]:
for(i in 1:4) {
    df[which(df[i, "sell_ct"] == i & df[i, "type"] == "Sell"), ]
    
    cat("Row number ", i, "type is :", df[i, "type"], " The Sell qty: ", df$quan[i], " number of shares. ", "\n")
    
    }
    

In [ ]:
# df[which(df$B == value_to_find), ]

# df[which(df$A == value_to_find_A & df$B == value_to_find_B), ]


for(i in 1:4) {
    df[which(df$sell_ct == i & df$type == "Sell"), ]
    
    if(df[i,"type"] == "Sell") {
    cat("Row number ", i, " Selling ", df$quan[i], " number of shares. ", "\n")
    }
}


In [ ]:
ct <- 0 

for(i in 1:4) {

    repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            return()
        }
        i <- i + 1
     }
}


  


In [ ]:
x <- 1
df_pl <- data.frame()
pl <- function(x) {
    
    sells[x, 'tally_b'] <<- buys[x, 'basis']
    
    sells[x, 'Sale'] <<- abs(sells[x, 'basis']) - sells[x, 'tally_b']
    temp <- data.frame(sells[x,])
    df_pl <<- rbind(df_pl, temp)
    
    }
 pl(x)
 head(df_pl, 3)
# head(sells, 3)
